DATA HANDLING


Importing data from NBA API:
* recent games - 2020-2021
* players rate

In this Notebook we will clean, format, and filter the data.



In [3]:
import pandas as pd
import csv
import os
import numpy as np

PLAYERS DATA

In [4]:
#Loading data 
df_players = pd.read_csv('./nba-players.csv')

#Printing columns
print('\nPlayers Data:')
print('\n','#' * 20)
print("\nColumns before data handling")
print('\n','-' * 5,'>', list(df_players.columns))


Players Data:

 ####################

Columns before data handling

 ----- > ['PLAYER_ID', 'RANK', 'PLAYER', 'TEAM', 'GP', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'EFF', 'AST_TOV', 'STL_TOV']


In [5]:
#Removing unnecessary columns
remove_col_players = ['PF','PTS','EFF','AST_TOV','STL_TOV','FG3_PCT','FT_PCT','FG_PCT']
df_players = df_players.drop(remove_col_players, axis = 1)


# Renaming columns
change_name_players = {
    'GP':'GAMES_PLAYED',
    'MIN':'MINUTES_PLAYED',
    'FGM':'TOTAL_THROWS_SCORED',
    'FGA':'TOTAL_THROWS_ATTEMPTS',
    'FG3M':'THREE_THROWS_SCORED',
    'FG3A':'THREE_THROWS_ATTEMPTS',
    'FTM':'FREE_THROWS_SCORED',
    'FTA':'FREE_THROWS_ATTEMPTS',
    'OREB':'OFFENSIVE_REBOUNDS',
    'DREB':'DEFFENSIVE_REBOUNDS',
    'AST':'ASSISTS',
    'STL':'STEALS',
    'BLK':'BLOCKS'}

df_players = df_players.rename(columns = change_name_players)

In [6]:
def calculate_player_score(player_row):
    # function for scoring players by the metric system shows below
    # https://coachingtoolbox.net/analytics/player-performance-grading-scale.html

    # -------------------------------------
    #     PERFORMANCE GRADE VALUES
    # -------------------------------------
    # (FGM) TOTAL POINTS SCORED +1
    # (FGA - FGM) TOTAL SHOTS MISSED -1
    # (FG3M) 3 POINT SHOTS MADE +3
    # (FG3A - FG3M) 3 POINT SHOTS MISSED -1.5
    # (FTM) FREE THROWS MADE +1
    # (FTA - FTM) FREE THROWS MISSED -1
    # (OREB) OFFENSIVE REBOUNDS (FT) +2
    # (AST) ASSISTS +2
    # (DREB) DEFENSIVE REBOUNDS (FT) +2
    # (STL) STEALS/RECOVERIES/INTERCEPTIONS +3
    # (BLK) BLOCKED SHOT +2


    # Creating Guidlines for player scoring
    score_guidlines = {
        "TOTAL_THROWS_SCORED":1,
        "TOTAL_THROWS_MISSED":-1,
        "THREE_THROWS_SCORED":3,
        "THREE_THROWS_MISSED":-1.5,
        "FREE_THROWS_SCORED":1,
        "FREE_THROWS_MISSED":-1,
        "OFFENSIVE_REBOUNDS":2,
        "DEFFENSIVE_REBOUNDS":2,
        "ASSISTS":2,
        "STEALS":3,
        "BLOCKS":2
    }

    #calculating the score of a player
    totalscore = 0
    for field,points in score_guidlines.items():
        totalscore = totalscore + ( player_row[field] * points )
    
    totalscore = totalscore / player_row['MINUTES_PLAYED']
    
    return totalscore*100




In [7]:
# Adding 'Missed Throws' Colums
df_players['TOTAL_THROWS_MISSED'] = df_players['TOTAL_THROWS_ATTEMPTS'] - df_players['TOTAL_THROWS_SCORED']
df_players['THREE_THROWS_MISSED'] = df_players['THREE_THROWS_ATTEMPTS'] - df_players['THREE_THROWS_SCORED']
df_players['FREE_THROWS_MISSED'] = df_players['FREE_THROWS_SCORED'] - df_players['FREE_THROWS_ATTEMPTS']


# Calculating Score
df_players['TOTAL_SCORE'] = calculate_player_score(df_players)

# Displaying Dataset
df_players.sort_values('TOTAL_SCORE')

,PLAYER_ID,RANK,PLAYER,TEAM,GAMES_PLAYED,MINUTES_PLAYED,TOTAL_THROWS_SCORED,TOTAL_THROWS_ATTEMPTS,THREE_THROWS_SCORED,THREE_THROWS_ATTEMPTS,...,DEFFENSIVE_REBOUNDS,REB,ASSISTS,STEALS,BLOCKS,TOV,TOTAL_THROWS_MISSED,THREE_THROWS_MISSED,FREE_THROWS_MISSED,TOTAL_SCORE
539,1630266,536,Will Magnay,NOP,1,3,0,1,0,1,...,0,0,0,0,0,1,1,1,0,-83.333333
537,204222,536,Greg Whittington,DEN,4,12,0,3,0,2,...,0,0,0,0,0,0,3,2,0,-50.000000
534,1629033,532,Theo Pinson,NYK,17,34,1,9,0,8,...,5,5,1,0,0,1,8,8,0,-20.588235
538,203943,536,Noah Vonleh,BKN,4,11,0,3,0,2,...,1,1,1,0,0,2,3,2,0,-18.181818
531,202697,532,Iman Shumpert,BKN,2,11,1,4,0,3,...,0,1,0,1,0,1,3,3,0,-13.636364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,1627780,482,Gary Payton II,GSW,10,40,10,13,2,4,...,9,11,1,6,1,1,3,2,-1,145.000000
197,2730,198,Dwight Howard,PHI,69,1196,178,303,5,20,...,390,580,61,30,62,112,125,15,-89,146.446488
18,201566,19,Russell Westbrook,WAS,65,2369,544,1238,86,273,...,641,750,763,89,23,312,694,187,-142,151.097509
2,203999,3,Nikola Jokic,DEN,72,2488,732,1293,92,237,...,575,780,599,95,48,222,561,145,-52,151.225884


GAMES DATA

In [8]:
#Loading data 
df_games = pd.read_csv('./nba-scores.csv')

#Printing columns
print('\nGames Data:')
print('\n','#' * 20)
print("\nColumns before data handling")
print('\n','-' * 5,'>', list(df_games.columns))


Games Data:

 ####################

Columns before data handling

 ----- > ['GAME_DATE_EST', 'GAME_SEQUENCE', 'GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY_NAME', 'TEAM_NAME', 'TEAM_WINS', 'TEAM_LOSSES', 'PTS_QTR1', 'PTS_QTR2', 'PTS_QTR3', 'PTS_QTR4', 'PTS_OT1', 'PTS_OT2', 'PTS_OT3', 'PTS_OT4', 'PTS_OT5', 'PTS_OT6', 'PTS_OT7', 'PTS_OT8', 'PTS_OT9', 'PTS_OT10', 'PTS', 'FG_PCT', 'FT_PCT', 'FG3_PCT', 'AST', 'REB', 'TOV']


In [9]:
#Removing unnecessary columns
remove_col_games = ['GAME_SEQUENCE','PTS_OT3','PTS_OT4','PTS_OT5','PTS_OT6','PTS_OT7','PTS_OT8','PTS_OT9','PTS_OT10']
df_games = df_games.drop(remove_col_games, axis = 1)



change_name_games = {
    'GAME_DATE_EST':'DATE',
    'TEAM_ABBREVIATION':'TEAM', 
    'TEAM_CITY_NAME':'CITY',
    'PTS_QTR1':'POINTS_Q1', 
    'PTS_QTR2':'POINTS_Q2', 
    'PTS_QTR3':'POINTS_Q3', 
    'PTS_QTR4':'POINTS_Q4', 
    'PTS':'TOTAL_POINTS', 
    'PTS_OT1':'OVERTIME1_POINTS', 
    'PTS_OT2':'OVERTIME2_POINTS'}

df_games = df_games.rename(columns = change_name_games)

In [10]:
df_players['TOTAL_SCORE'].mean()

73.61713678064152

NEW TABLES

In [11]:
# Lets create new df of nba teams

def calculate_team_total_score(team_name):
    team_players = df_players.loc[df_players['TEAM'] == team_name]
    return team_players['TOTAL_SCORE'].mean()

team_names = df_players['TEAM'].unique()
total_scores = []
for team in team_names:
    total_scores.append(calculate_team_total_score(team))

df_teams = pd.DataFrame(columns=['TEAM_NAME','TEAM_ID','TOTAL_TEAM_SCORE'])
df_teams['TEAM_NAME'] = team_names
df_teams['TEAM_ID'] = range(1,len(team_names)+1)
df_teams['TOTAL_TEAM_SCORE'] = total_scores



In [12]:
df_games_united = pd.DataFrame(columns=['DATE','GAME_ID','TEAM_A','TEAM_A_ID','TEAM_A_SCORE','TEAM_B','TEAM_B_ID','TEAM_B_SCORE','TEAM_A_POINTS','TEAM_B_POINTS'])
unique_games_id = df_games['GAME_ID'].unique()

In [13]:
def unite_game_rows(game_id):
    games = df_games.loc[df_games['GAME_ID'] == game_id]
    #print(f"working on {game_id}")
    
    DATE = games.iloc[0]['DATE']
    GAME_ID = game_id
    
    TEAM_A = games.iloc[0]['TEAM']
    TEAM_A_ID = int(df_teams.loc[df_teams['TEAM_NAME'] == TEAM_A]['TEAM_ID'])
    TEAM_A_SCORE = float(df_teams.loc[df_teams['TEAM_NAME'] == TEAM_A]['TOTAL_TEAM_SCORE'])
    TEAM_A_POINTS = games.iloc[0]['TOTAL_POINTS']

    TEAM_B = games.iloc[1]['TEAM']
    TEAM_B_ID = int(df_teams.loc[df_teams['TEAM_NAME'] == TEAM_B]['TEAM_ID'])
    TEAM_B_SCORE = float(df_teams.loc[df_teams['TEAM_NAME'] == TEAM_B]['TOTAL_TEAM_SCORE'])
    TEAM_B_POINTS = games.iloc[1]['TOTAL_POINTS']
    
    return pd.Series([DATE,GAME_ID,TEAM_A,TEAM_A_ID,TEAM_A_SCORE,TEAM_B,TEAM_B_ID,TEAM_B_SCORE,TEAM_A_POINTS,TEAM_B_POINTS], index = df_games_united.columns)


unique_games_ids = df_games['GAME_ID'].unique()
for game_id in unique_games_ids:
    row = unite_game_rows(game_id)    
    df_games_united = df_games_united.append(row, ignore_index=True)    

In [23]:
#Adding win_ratio to the team dataframe
win_ratio_array = []
for team in df_teams['TEAM_NAME']:
    query = f"TEAM_A == '{team}' or TEAM_B == '{team}'"
    team_games = df_games_united.query(query)
    win_count = 0
    for row in team_games.iterrows():
        if row[1]['TEAM_A'] == team and row[1]['TEAM_A_POINTS'] > row[1]['TEAM_B_POINTS']:
                win_count = win_count + 1
        elif row[1]['TEAM_B'] == team and row[1]['TEAM_A_POINTS'] < row[1]['TEAM_B_POINTS']:
                win_count = win_count + 1

    print(f"{team} : {win_count}/{team_games.shape[0]}")
    win_ratio_array.append(win_count/team_games.shape[0])

df_teams['WIN_RATIO'] = win_ratio_array

GSW : 56/142
POR : 82/165
DEN : 109/180
WAS : 62/157
DAL : 94/166
MIL : 117/172
PHX : 97/164
NYK : 66/147
BOS : 97/172
NOP : 66/149
CHI : 56/141
ATL : 71/155
SAC : 65/151
CLE : 43/141
PHI : 102/166
BKN : 93/165
CHA : 57/142
UTA : 110/168
MIN : 43/139
SAS : 66/150
LAC : 113/178
IND : 82/157
DET : 42/142
MEM : 79/160
MIA : 100/175
TOR : 91/161
LAL : 119/178
HOU : 71/163
OKC : 73/157
ORL : 58/157


In [24]:
df_teams

,TEAM_NAME,TEAM_ID,TOTAL_TEAM_SCORE,WIN_RATIO
0,GSW,1,85.100871,0.394366
1,POR,2,77.225285,0.496970
2,DEN,3,66.692896,0.605556
3,WAS,4,72.563369,0.394904
4,DAL,5,69.965595,0.566265
5,MIL,6,73.420374,0.680233
6,PHX,7,73.107778,0.591463
7,NYK,8,66.012107,0.448980
8,BOS,9,80.303052,0.563953
9,NOP,10,64.015418,0.442953


In [25]:
team_a_ratios = []
team_b_ratios = []
for game in df_games_united.iterrows():
    team_a_ratios.append(df_teams.loc[df_teams['TEAM_NAME'] == game[1]['TEAM_A']].iloc[0]['WIN_RATIO'])
    team_b_ratios.append(df_teams.loc[df_teams['TEAM_NAME'] == game[1]['TEAM_B']].iloc[0]['WIN_RATIO'])

df_games_united['TEAM_A_WIN_RATIO'] = team_a_ratios
df_games_united['TEAM_B_WIN_RATIO'] = team_b_ratios

df_games_united


,DATE,GAME_ID,TEAM_A,TEAM_A_ID,TEAM_A_SCORE,TEAM_B,TEAM_B_ID,TEAM_B_SCORE,TEAM_A_POINTS,TEAM_B_POINTS,TEAM_A_WIN_RATIO,TEAM_B_WIN_RATIO
0,22/10/2019,21900001,NOP,10,64.015418,TOR,26,73.806966,122,130,0.442953,0.565217
1,22/10/2019,21900002,LAL,27,82.805261,LAC,21,77.637377,102,112,0.668539,0.634831
2,23/10/2019,21900003,CHI,11,73.896496,CHA,17,72.331786,125,126,0.397163,0.401408
3,23/10/2019,21900004,DET,23,73.043957,IND,22,74.117156,119,110,0.295775,0.522293
4,23/10/2019,21900005,CLE,14,74.998370,ORL,30,67.977461,85,94,0.304965,0.369427
...,...,...,...,...,...,...,...,...,...,...,...,...
2375,18/06/2021,42000226,UTA,18,75.414780,LAC,21,77.637377,119,131,0.654762,0.634831
2376,19/06/2021,42000217,MIL,6,73.420374,BKN,16,74.287874,115,111,0.680233,0.563636
2377,20/06/2021,42000311,LAC,21,77.637377,PHX,7,73.107778,114,120,0.634831,0.591463
2378,20/06/2021,42000207,ATL,12,74.175677,PHI,15,71.085154,103,96,0.458065,0.614458


In [14]:
#Data shape
print('\nShape of players data:')
print('\n','-' * 10,df_players.shape)
print('\nShape of games data:')
print('\n','-' * 10,df_games.shape)
print('\nShape of teams data:')
print('\n','-' * 10,df_teams.shape)
print('\nShape of games united data:')
print('\n','-' * 10,df_games_united.shape)


Shape of players data:

 ---------- (540, 23)

Shape of games data:

 ---------- (4760, 21)

Shape of teams data:

 ---------- (30, 3)

Shape of games united data:

 ---------- (2380, 10)


In [15]:
#Export
df_players.to_csv('./player-data.csv',index=False)
df_games.to_csv('./games-data.csv',index=False)
df_teams.to_csv('./teams-data.csv',index=False)
df_games_united.to_csv('./games-united.csv',index=False)

SHOW DATA

In [16]:
df_players

,PLAYER_ID,RANK,PLAYER,TEAM,GAMES_PLAYED,MINUTES_PLAYED,TOTAL_THROWS_SCORED,TOTAL_THROWS_ATTEMPTS,THREE_THROWS_SCORED,THREE_THROWS_ATTEMPTS,...,DEFFENSIVE_REBOUNDS,REB,ASSISTS,STEALS,BLOCKS,TOV,TOTAL_THROWS_MISSED,THREE_THROWS_MISSED,FREE_THROWS_MISSED,TOTAL_SCORE
0,201939,1,Stephen Curry,GSW,63,2152,658,1365,337,801,...,316,345,363,77,8,213,707,464,-33,107.992565
1,203081,2,Damian Lillard,POR,67,2398,602,1334,275,704,...,251,283,505,62,17,203,732,429,-35,97.226856
2,203999,3,Nikola Jokic,DEN,72,2488,732,1293,92,237,...,575,780,599,95,48,222,561,145,-52,151.225884
3,203078,4,Bradley Beal,WAS,60,2147,670,1382,130,373,...,211,283,265,69,22,187,712,243,-51,83.348859
4,1629029,5,Luka Doncic,DAL,66,2262,647,1351,192,548,...,473,527,567,64,36,281,704,356,-127,128.558798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,1628394,536,Anzejs Pasecniks,WAS,1,6,0,1,0,1,...,0,1,1,0,0,5,1,1,0,25.000000
536,1630204,536,Ashton Hagans,MIN,2,4,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0.000000
537,204222,536,Greg Whittington,DEN,4,12,0,3,0,2,...,0,0,0,0,0,0,3,2,0,-50.000000
538,203943,536,Noah Vonleh,BKN,4,11,0,3,0,2,...,1,1,1,0,0,2,3,2,0,-18.181818


In [17]:
df_games

,DATE,GAME_ID,TEAM_ID,TEAM,CITY,TEAM_NAME,TEAM_WINS,TEAM_LOSSES,POINTS_Q1,POINTS_Q2,...,POINTS_Q4,OVERTIME1_POINTS,OVERTIME2_POINTS,TOTAL_POINTS,FG_PCT,FT_PCT,FG3_PCT,AST,REB,TOV
0,22/10/2019,21900001,1610612740,NOP,New Orleans,Pelicans,0,1,30,31,...,31,5,0,122,0.422,0.850,0.422,30,53,19
1,22/10/2019,21900001,1610612761,TOR,Toronto,Raptors,1,0,27,29,...,29,13,0,130,0.408,0.842,0.350,23,57,16
2,22/10/2019,21900002,1610612747,LAL,Los Angeles,Lakers,0,1,25,29,...,17,0,0,102,0.435,0.714,0.394,20,41,14
3,22/10/2019,21900002,1610612746,LAC,LA,Clippers,1,0,22,40,...,27,0,0,112,0.519,0.708,0.355,24,45,14
4,23/10/2019,21900003,1610612741,CHI,Chicago,Bulls,0,1,28,27,...,30,0,0,125,0.467,0.818,0.300,25,49,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4755,20/06/2021,42000311,1610612756,PHX,Phoenix,Suns,1,0,21,36,...,27,0,0,120,0.551,1.000,0.406,31,43,7
4756,20/06/2021,42000207,1610612737,ATL,Atlanta,Hawks,4,3,25,23,...,27,0,0,103,0.444,0.750,0.259,22,44,10
4757,20/06/2021,42000207,1610612755,PHI,Philadelphia,76ers,3,4,28,18,...,25,0,0,96,0.425,0.760,0.321,26,48,17
4758,22/06/2021,42000312,1610612746,LAC,LA,Clippers,0,1,22,25,...,32,0,0,103,0.450,0.750,0.382,19,39,8


In [18]:
df_teams

,TEAM_NAME,TEAM_ID,TOTAL_TEAM_SCORE
0,GSW,1,85.100871
1,POR,2,77.225285
2,DEN,3,66.692896
3,WAS,4,72.563369
4,DAL,5,69.965595
5,MIL,6,73.420374
6,PHX,7,73.107778
7,NYK,8,66.012107
8,BOS,9,80.303052
9,NOP,10,64.015418


In [19]:
df_games_united

,DATE,GAME_ID,TEAM_A,TEAM_A_ID,TEAM_A_SCORE,TEAM_B,TEAM_B_ID,TEAM_B_SCORE,TEAM_A_POINTS,TEAM_B_POINTS
0,22/10/2019,21900001,NOP,10,64.015418,TOR,26,73.806966,122,130
1,22/10/2019,21900002,LAL,27,82.805261,LAC,21,77.637377,102,112
2,23/10/2019,21900003,CHI,11,73.896496,CHA,17,72.331786,125,126
3,23/10/2019,21900004,DET,23,73.043957,IND,22,74.117156,119,110
4,23/10/2019,21900005,CLE,14,74.998370,ORL,30,67.977461,85,94
...,...,...,...,...,...,...,...,...,...,...
2375,18/06/2021,42000226,UTA,18,75.414780,LAC,21,77.637377,119,131
2376,19/06/2021,42000217,MIL,6,73.420374,BKN,16,74.287874,115,111
2377,20/06/2021,42000311,LAC,21,77.637377,PHX,7,73.107778,114,120
2378,20/06/2021,42000207,ATL,12,74.175677,PHI,15,71.085154,103,96


In [20]:
#Columns type
print('Columns data type:')
print('\n','#' * 20)
print('\nPlayers Data:')
print('\n','-' * 10,'\n', df_players.dtypes)
print('\n\nGames Data:')
print('\n','-' * 10,'\n', df_games.dtypes)
print('\n\nTeams Data:')
print('\n','-' * 10,'\n', df_teams.dtypes)
print('\n\nGames United Data:')
print('\n','-' * 10,'\n', df_games_united.dtypes)

Columns data type:

 ####################

Players Data:

 ---------- 
 PLAYER_ID                  int64
RANK                       int64
PLAYER                    object
TEAM                      object
GAMES_PLAYED               int64
MINUTES_PLAYED             int64
TOTAL_THROWS_SCORED        int64
TOTAL_THROWS_ATTEMPTS      int64
THREE_THROWS_SCORED        int64
THREE_THROWS_ATTEMPTS      int64
FREE_THROWS_SCORED         int64
FREE_THROWS_ATTEMPTS       int64
OFFENSIVE_REBOUNDS         int64
DEFFENSIVE_REBOUNDS        int64
REB                        int64
ASSISTS                    int64
STEALS                     int64
BLOCKS                     int64
TOV                        int64
TOTAL_THROWS_MISSED        int64
THREE_THROWS_MISSED        int64
FREE_THROWS_MISSED         int64
TOTAL_SCORE              float64
dtype: object


Games Data:

 ---------- 
 DATE                 object
GAME_ID               int64
TEAM_ID               int64
TEAM                 object
CITY          

In [21]:
#Empty/NaN cells
print('\nEmpty cells:\n')
print('\n','#' * 20)
print('\nPlayers Data:')     
print('\n','-' * 10,'\n', df_players.isnull().values.any())   
print('\n\nGames Data:')   
print('\n','-' * 10,'\n', df_games.isnull().values.any())  
print('\n\nTeams Data:')   
print('\n','-' * 10,'\n', df_teams.isnull().values.any())  
print('\n\nGames United Data:')   
print('\n','-' * 10,'\n', df_games_united.isnull().values.any()) 


Empty cells:


 ####################

Players Data:

 ---------- 
 False


Games Data:

 ---------- 
 False


Teams Data:

 ---------- 
 False


Games United Data:

 ---------- 
 False


In [22]:
#Number of empty/NaN cells
print('\nNumber of Empty cells:')
print('\n','#' * 20)
print('\nPlayers Data:')
print('\n','-' * 10,'\n', df_players.isnull().sum())
print('\n\nGames Data:')           
print('\n','-' * 10,'\n', df_games.isnull().sum())
print('\n\nTeams Data:')           
print('\n','-' * 10,'\n', df_teams.isnull().sum())
print('\n\nGames United Data:')           
print('\n','-' * 10,'\n', df_games_united.isnull().sum())


Number of Empty cells:

 ####################

Players Data:

 ---------- 
 PLAYER_ID                0
RANK                     0
PLAYER                   0
TEAM                     0
GAMES_PLAYED             0
MINUTES_PLAYED           0
TOTAL_THROWS_SCORED      0
TOTAL_THROWS_ATTEMPTS    0
THREE_THROWS_SCORED      0
THREE_THROWS_ATTEMPTS    0
FREE_THROWS_SCORED       0
FREE_THROWS_ATTEMPTS     0
OFFENSIVE_REBOUNDS       0
DEFFENSIVE_REBOUNDS      0
REB                      0
ASSISTS                  0
STEALS                   0
BLOCKS                   0
TOV                      0
TOTAL_THROWS_MISSED      0
THREE_THROWS_MISSED      0
FREE_THROWS_MISSED       0
TOTAL_SCORE              0
dtype: int64


Games Data:

 ---------- 
 DATE                0
GAME_ID             0
TEAM_ID             0
TEAM                0
CITY                0
TEAM_NAME           0
TEAM_WINS           0
TEAM_LOSSES         0
POINTS_Q1           0
POINTS_Q2           0
POINTS_Q3           0
POINTS_Q4         

END